In [25]:
#start
import imaplib
import email
import json

#read credentials from json
with open("ym_creds.json", "r") as f:
    jmCreds = json.load(f)

In [26]:
# Connect to the Yahoo Mail IMAP server
imap_server = imaplib.IMAP4_SSL("imap.mail.yahoo.com")
# Login to the account
imap_server.login(jmCreds["email"], jmCreds["password"])
# Select the "Inbox" folder
imap_server.select("Inbox")

('OK', [b'5520'])

In [27]:
# Search for all emails
status, email_ids = imap_server.search(None, "ALL")
# Search for all emails from MAXIMA
status, email_ids = imap_server.search(None, "FROM noreply.code.provider@maxima.lt")
# Print the list of email IDs
print(email_ids)

[b'2674 2826 3711 3712 3730 3754 3770 3780 3781 3792 3793 3797 3805 3812 3814 3858 3876 3894 3906 3907 3929 3944 3946 3956 3958 3972 3979 3980 3983 3984 3985 3986 3990 4007 4009 4020 4026 4027 4036 4045 4051 4055 4075 4082 4085 4090 4120 4128 4136 4145 4160 4180 4203 4204 4211 4224 4234 4246 4250 4254 4263 4264 4271 4272 4286 4289 4291 4300 4303 4314 4319 4320 4335 4342 4348 4349 4351 4365 4367 4391 4408 4419 4428 4437 4440 4453 4476 4481 4503 4506 4518 4528 4534 4546 4552 4602 4603 4608 4615 4630 4631 4640 4653 4657 4678 4693 4696 4703 4708 4721 4737 4738 4745 4748 4755 4756 4757 4778 4785 4793 4802 4812 4819 4826 4846 4856 4859 4861 4883 4888 4891 4892 4899 4910 4919 4933 4938 4942 4945 4956 4960 4969 4983 4986 4987 5001 5002 5010 5013 5019 5022 5023 5025 5029 5030 5034 5035 5047 5048 5049 5056 5064 5065 5085 5087 5106 5133 5135 5144 5165 5169 5184 5185 5188 5191 5192 5194 5232 5235 5247 5250 5253 5256 5283 5289 5293 5304 5305 5306 5307 5316 5334 5346 5354 5362 5378 5401 5403 5405 54

In [28]:
#get message bu ID
status, email_data = imap_server.fetch("5495", "(RFC822)")
email_message = email.message_from_bytes(email_data[0][1])

In [29]:
msg1 = email_message.get_payload()[0]
msg1body = msg1.get_payload(decode=True)
msg2 = email_message.get_payload()[0]
msg2body = msg2.get_payload(decode=True)
msg3 = email_message.get_payload()[0]
msg3body = msg3.get_payload(decode=True)

In [31]:
# print(msg1.get_payload(decode=True), msg2.get_payload(decode=True))
print(msg2body.decode())

<html><body>


<html>

<body>

    <div class="previewText" style="display:none; max-height: 0px; overflow: hidden">
            <p>Apsipirkimo suma: 6.08 EUR</p>
            <br />
                <p>Kvito nuolaidų suma: 0.59 EUR</p>
                <br />
                <p> MAXIMOS PINIGŲ likutis: 0.06 EUR </p>
            <p>
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&

In [9]:

# Check if the email is a multipart email
if email_message.is_multipart():
    # Loop through the message parts
    for part in email_message.get_payload():
        # Decode the body of each part
        decoded_body = part.get_payload(decode=True)
        # Print the decoded body
        print(decoded_body.decode())
else:
    # Decode the body of the email
    decoded_body = email_message.get_payload(decode=True)
    # Print the decoded body
    print(decoded_body.decode())

<html><body>


<html>

<body>

    <div class="previewText" style="display:none; max-height: 0px; overflow: hidden">
            <p>Apsipirkimo suma: 13.48 EUR</p>
            <br />
                <p>Kvito nuolaidų suma: 3.08 EUR</p>
                <br />
                <p> MAXIMOS PINIGŲ likutis: 0.13 EUR </p>
            <p>
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfe in position 2: invalid start byte

In [32]:
# Disconnect from the server
imap_server.close()
imap_server.logout()

('BYE', [b'IMAP4rev1 Server logging out'])